Code

https://github.com/nicknochnack/OpenAI-Reinforcement-Learning-with-Custom-Environment

Video

https://www.youtube.com/redirect?event=video_description&redir_token=QUFFLUhqbk1Iang2ME0wODgzTlctRVl6LUdBRFU0WFloZ3xBQ3Jtc0tua2k5cFpNbzlNX2UxWmlzXzl5MzlZX3VPd0RoZ1Y3UVpEUjl5MWkweWs4TFM2MmpYWUdESEI0VmU2TndCVkJUaWFUUmZraVFGN3p5WWdyalRzOFhxcmdfU2tRSkwzdGlyMDN3eGFVRVh5RWhQWklicw&q=https%3A%2F%2Fgithub.com%2Fnicknochnack%2FOpenAI-Reinforcement-Learning-with-Custom-Environment&v=bD6V3rcr_54

# 0. Install Dependencies

In [13]:
# !pip install tensorflow
#!pip install gym
#!pip install keras
!pip install keras-rl2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 657.5 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 12.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 30.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 35.7 MB/s eta 0:00:0000:0100:01m
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.3.0
    Uninstalling typing_extensions-4.3.0:
      Successfully uninstalled typing_extensions-4.3.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: h5py
    Found existing installation: h5py 3.7.0
    Uninstalling h5py-3.7.0:
      Successfully uninstalled h5py-3.7.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.10.0
    Uninstalling tensorboard-2.10.0:
      Successfully uninst

# 1. Test Random Environment with OpenAI Gym

In [1]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

In [2]:
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set start temp
        self.state = 38 + random.randint(-3,3)
        # Set shower length
        self.shower_length = 60
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 
        # 2 -1 = 1 temperature 
        self.state += action -1 
        # Reduce shower length by 1 second
        self.shower_length -= 1 
        
        # Calculate reward
        if self.state >=37 and self.state <=39: 
            reward =1 
        else: 
            reward = -1 
        
        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = 38 + random.randint(-3,3)
        # Reset shower time
        self.shower_length = 60 
        return self.state
    

In [3]:
env = ShowerEnv()

/opt/conda/lib/python3.7/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [4]:
env.observation_space.sample()

array([12.060718], dtype=float32)

In [5]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        # env.render(mode="ipython")
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:-22
Episode:2 Score:-16
Episode:3 Score:-22
Episode:4 Score:-4
Episode:5 Score:-36
Episode:6 Score:-38
Episode:7 Score:-38
Episode:8 Score:-26
Episode:9 Score:-58
Episode:10 Score:-52


# 2. Create a Deep Learning Model with Keras

In [33]:
import numpy as np
import tensorflow.keras as keras

from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.optimizers import adam_v2

In [23]:
states = env.observation_space.shape
actions = env.action_space.n

In [24]:
states

(1,)

In [25]:
def build_model(states, actions):
    model = Sequential()    
    model.add(Dense(24, activation='relu', input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [26]:
# del model 

In [27]:
model = build_model(states, actions)

In [28]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 24)                48        
_________________________________________________________________
dense_4 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 75        
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


# 3. Build Agent with Keras-RL

In [29]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [30]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [34]:
dqn = build_agent(model, actions)

In [35]:
dqn.compile(adam_v2.Adam(learning_rate=1e-3), metrics=['mae'])

2022-10-06 16:25:12.130564: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [36]:
dqn.fit(env, nb_steps=5000, visualize=False, verbose=1)

Training for 5000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 25:14 - reward: -1.0000

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
/opt/conda/lib/python3.7/site-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


 4998/10000 [=============>................] - ETA: 47s - reward: -0.4094done, took 47.318 seconds


In [37]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 60.000, steps: 60
Episode 2: reward: -60.000, steps: 60
Episode 3: reward: 60.000, steps: 60
Episode 4: reward: -60.000, steps: 60
Episode 5: reward: -60.000, steps: 60
Episode 6: reward: -60.000, steps: 60
Episode 7: reward: 60.000, steps: 60
Episode 8: reward: -60.000, steps: 60
Episode 9: reward: 60.000, steps: 60
Episode 10: reward: -60.000, steps: 60
Episode 11: reward: -60.000, steps: 60
Episode 12: reward: 60.000, steps: 60
Episode 13: reward: -60.000, steps: 60
Episode 14: reward: -60.000, steps: 60
Episode 15: reward: -60.000, steps: 60
Episode 16: reward: -60.000, steps: 60
Episode 17: reward: 60.000, steps: 60
Episode 18: reward: -60.000, steps: 60
Episode 19: reward: 60.000, steps: 60
Episode 20: reward: -60.000, steps: 60
Episode 21: reward: -60.000, steps: 60
Episode 22: reward: -60.000, steps: 60
Episode 23: reward: -60.000, steps: 60
Episode 24: reward: 60.000, steps: 60
Episode 25: reward: -60.000, steps: 60
Episode 26: r

In [38]:
_ = dqn.test(env, nb_episodes=15, visualize=False)

Testing for 15 episodes ...
Episode 1: reward: -60.000, steps: 60
Episode 2: reward: -60.000, steps: 60
Episode 3: reward: -60.000, steps: 60
Episode 4: reward: 60.000, steps: 60
Episode 5: reward: 60.000, steps: 60
Episode 6: reward: -60.000, steps: 60
Episode 7: reward: -60.000, steps: 60
Episode 8: reward: 60.000, steps: 60
Episode 9: reward: 60.000, steps: 60
Episode 10: reward: 60.000, steps: 60
Episode 11: reward: -60.000, steps: 60
Episode 12: reward: 60.000, steps: 60
Episode 13: reward: 60.000, steps: 60
Episode 14: reward: 60.000, steps: 60
Episode 15: reward: -60.000, steps: 60


# 4. Reloading Agent from Memory

In [39]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [ ]:
del model
del dqn
del env

In [43]:
# env = gym.make('CartPole-v0')
env = ShowerEnv()
actions = env.action_space.n
states = env.observation_space.shape
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(adam_v2.Adam(learning_rate=1e-3), metrics=['mae'])

/opt/conda/lib/python3.7/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [44]:
dqn.load_weights('dqn_weights.h5f')

In [46]:
_ = dqn.test(env, nb_episodes=5, visualize=False)

Testing for 5 episodes ...
Episode 1: reward: -60.000, steps: 60
Episode 2: reward: 60.000, steps: 60
Episode 3: reward: -60.000, steps: 60
Episode 4: reward: -60.000, steps: 60
Episode 5: reward: 60.000, steps: 60
